<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загружаем-данные" data-toc-modified-id="Загружаем-данные-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загружаем данные</a></span></li><li><span><a href="#func" data-toc-modified-id="func-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>func</a></span></li><li><span><a href="#features" data-toc-modified-id="features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>features</a></span></li><li><span><a href="#fit" data-toc-modified-id="fit-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>fit</a></span></li><li><span><a href="#baseline-cv" data-toc-modified-id="baseline-cv-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>baseline cv</a></span></li><li><span><a href="#sklearn-api" data-toc-modified-id="sklearn-api-6"><span class="toc-item-num">6&nbsp;&nbsp;</span><del>sklearn api</del></a></span></li><li><span><a href="#predict" data-toc-modified-id="predict-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>predict</a></span></li><li><span><a href="#save" data-toc-modified-id="save-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>save</a></span></li></ul></div>

In [18]:
import pandas as pd

from xgboost import XGBRegressor
import xgboost as xgb

# Загружаем данные

In [176]:
train_pool = pd.read_csv('train.csv', encoding='utf-8').sort_values('query_id')
test_pool = pd.read_csv('test.csv', encoding='utf-8').sort_values('query_id')

# func

Напишем функции, генерирующие простые признаки основанные на пересечении триграмм между запросом и названием организации

In [177]:
import metrics as m
import numpy as np
def ndcg(y_true, y_pred, groups):
    return np.nanmean([m.ndcg_score(np.array(y_true)[np.array(groups) == g], 
                                 np.array(y_pred)[np.array(groups) == g]) for g in set(groups)])

In [178]:
def get_trigrams(string):
    string = '^^' + string + '$$'
    trigrams = set()
    trigrams_count = 0
    
    for i in range(len(string) - 2):
        trigrams.add(string[i:i+3])
        trigrams_count += 1
        
    return trigrams, trigrams_count

def common_trigrams_factors(query, org_name):
    query_trigrams, query_trigrams_count = get_trigrams(query)
    org_name_trigrams, org_name_trigrams_count = get_trigrams(org_name)

    factors = [float(len(query_trigrams.intersection(org_name_trigrams)))]

    factors.append(0. if query_trigrams_count == 0. else 0.1 + factors[0] / query_trigrams_count)
    factors.append(0. if org_name_trigrams_count == 0. else 0.1 + factors[0] / org_name_trigrams_count)    
    
    return factors

In [179]:
def get_bigrams(string):
    string = '^^' + string + '$$'
    bigrams = set()
    bigrams_count = 0
    
    for i in range(len(string) - 1):
        bigrams.add(string[i:i+2])
        bigrams_count += 1
        
    return bigrams, bigrams_count

def common_bigrams_factors(query, org_name):
    query_bigrams, query_bigrams_count = get_bigrams(query)
    org_name_bigrams, org_name_bigrams_count = get_bigrams(org_name)

    factors = [float(len(query_bigrams.intersection(org_name_bigrams)))]

    factors.append(0. if query_bigrams_count == 0. else 0.1 + factors[0] / query_bigrams_count)
    factors.append(0. if org_name_bigrams_count == 0. else 0.1 + factors[0] / org_name_bigrams_count)    
    
    return factors

# features

Посчитаем данные факторы для каждого файла

In [180]:
train_factors

,0,1,2,0,1,2
0,17.0,0.323684,0.469565,9.0,0.220000,0.300000
19,14.0,0.284211,0.600000,7.0,0.193333,0.359259
18,10.0,0.231579,0.688235,5.0,0.166667,0.412500
17,20.0,0.363158,0.500000,12.0,0.260000,0.344898
15,19.0,0.350000,0.458491,12.0,0.260000,0.330769
14,20.0,0.363158,0.500000,14.0,0.286667,0.385714
13,26.0,0.442105,0.665217,18.0,0.340000,0.500000
12,11.0,0.244737,0.523077,4.0,0.153333,0.260000
11,21.0,0.376316,0.520000,12.0,0.260000,0.344898
10,8.0,0.205263,0.375862,4.0,0.153333,0.242857


In [181]:
train_group_sizes = train_pool.query_id.value_counts()

train_pool = train_pool.join(train_group_sizes.to_frame('group_sizes'), on='query_id')

In [182]:
test_group_sizes = test_pool.query_id.value_counts()

test_pool = test_pool.join(test_group_sizes.to_frame('group_sizes'), on='query_id')

In [183]:
def calc_trigram_factors(row):
    return pd.Series(common_trigrams_factors(row.query, row.org_name))
def calc_bigram_factors(row):
    return pd.Series(common_bigrams_factors(row.query, row.org_name))
    
train_factors = pd.concat((train_pool.apply(calc_bigram_factors, axis=1), 
                           train_pool.apply(calc_trigram_factors, axis=1)), axis=1)
test_factors = pd.concat((test_pool.apply(calc_bigram_factors, axis=1), 
                           test_pool.apply(calc_trigram_factors, axis=1)), axis=1)

# fit

Обучаем модель

In [184]:
clf = XGBRegressor(n_estimators=100)
clf.fit(train_factors.values, train_pool[['relevance']].values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

# baseline cv

In [157]:
from sklearn.model_selection import cross_val_predict, GroupKFold, cross_val_score

In [158]:
clf = XGBRegressor(n_estimators=100)
baseline_pred = cross_val_predict(
    clf,
    train_factors.values,
    train_pool[['relevance']].values,
    groups=train_pool.query_id.values,
    cv=5)
ndcg(train_pool.relevance.values, baseline_pred, train_pool.query_id.values)

0.7112039519725594

In [123]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
lr_pred = cross_val_predict(
    clf,
    X=train_factors.values,
    y=train_pool[['relevance']].values,
    groups=train_pool.query_id.values,
    cv=5)
ndcg(train_pool.relevance.values, lr_pred, train_pool.query_id.values)

D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


4.6456152873866445

In [124]:
from sklearn.svm import SVR
clf = SVR()
svr_pred = cross_val_predict(
    clf,
    X=train_factors.values,
    y=train_pool[['relevance']].values,
    groups=train_pool.query_id.values,
    cv=5)
ndcg(train_pool.relevance.values, svr_pred, train_pool.query_id.values)

c:\users\nadzeya_salahub\appdata\local\continuum\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nadzeya_salahub\appdata\local\continuum\anaconda3\envs\py36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\nadzeya_salahub\appdata\local\continuum\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\nadzeya_salahub\appdat

0.6761102646955143

# ~~sklearn api~~

In [197]:
kf = GroupKFold(4)

In [281]:
from sklearn.base import clone

def cross_val_predict_custom(train_pool=train_pool, train_factors=train_factors, cv_n=5, 
                             params={'objective': 'rank:pairwise', 'silent': True}):
    kf = GroupKFold(cv_n)
    cv_preds = np.zeros(train_pool.shape[0])
    for tr_i, te_i in kf.split(train_factors.values, train_pool.relevance.values, train_pool.query_id.values):
        print(tr_i)
        tr_i = sorted(tr_i)
        te_i = sorted(te_i)
        
        train_queries = train_pool.query_id.iloc[tr_i].drop_duplicates().values
        test_queries = train_pool.query_id.iloc[te_i].drop_duplicates().values
        
        train_dmatrix = xgb.DMatrix(train_factors.values[tr_i], train_pool.relevance.values[tr_i])
        train_dmatrix.set_group(train_group_sizes.loc[train_queries].values)

        test_dmatrix = xgb.DMatrix(train_factors.values[te_i])
        test_dmatrix.set_group(train_group_sizes.loc[test_queries].values)
        
        regr = xgb.train(params, train_dmatrix, 50)
        cv_preds[te_i] = regr.predict(test_dmatrix)
    return cv_preds, params

In [282]:
cv_p, params = cross_val_predict_custom(params={
    'objective': 'rank:pairwise', 
    'silent': True,
    'max_depth': 3
})

[    0     1     2 ... 29245 29246 29247]
[    0     1     2 ... 29271 29272 29273]
[   20    21    22 ... 29271 29272 29273]
[    0     1     2 ... 29271 29272 29273]
[    0     1     2 ... 29271 29272 29273]


In [283]:
ndcg(train_pool.relevance.values, cv_p, train_pool.query_id.values)

D:\Courses\shad2018\ML\competition2\metrics.py:147: RuntimeWarning: invalid value encountered in double_scalars
  return actual / best


0.74678576256303

# predict

In [284]:
train_dmatrix = xgb.DMatrix(train_factors.values, train_pool.relevance)
train_dmatrix.set_group(train_group_sizes.values)

In [285]:
test_dmatrix = xgb.DMatrix(test_factors.values)
test_dmatrix.set_group(test_group_sizes.values)

In [286]:
clf = xgb.train(params, train_dmatrix, 50)
predictions = clf.predict(test_dmatrix)
test_pool['relevance'] = predictions

# save

Сортируем организации по предсказанной релевантности для каждой организации и записываем их в файл

In [287]:
test_pool.sort_values(['query_id', 'relevance'], ascending=[True, False])[['query_id', 'org_id']].to_csv('submission1.csv', index=None)

In [290]:
params

{'objective': 'rank:pairwise', 'silent': True, 'max_depth': 3}